In [ ]:
import yaml
import pickle
import os.path as osp
import subprocess
from datetime import timedelta
from urllib.parse import urlparse
import numpy as np
import matplotlib.pyplot as plt
from moisture_rnn_pkl import pkl2train
import time
from utils import time_intp, str2time, filter_nan_values, read_pkl, read_yml, retrieve_url

## Setup

In [ ]:
retrieve_url("https://demo.openwfm.org/web/data/fmda/dicts/fmda_nw_202401-05_f05.pkl", "data/fmda_nw_202401-05_f05.pkl")

In [ ]:
data_params = read_yml("params_data.yaml")
data_params

In [ ]:
# dat = read_pkl("data/test_CA_202401.pkl")
dat = read_pkl("data/fmda_nw_202401-05_f05.pkl")

## Format

In [ ]:
train = pkl2train(["data/fmda_nw_202401-05_f05.pkl"])

In [ ]:
train['PLFI1_202401'].keys()

In [ ]:
from utils import hash_ndarray

hash_ndarray(train['PLFI1_202401']['X'])

In [ ]:
hash_ndarray(train['YTCW1_202401']['X'])

In [ ]:
len(train.keys())

In [ ]:
dat = read_pkl("data/fmda_nw_202401-05_f05.pkl")

In [ ]:
import importlib
import data_funcs
importlib.reload(data_funcs)
from data_funcs import build_train_dict

In [ ]:
# train = build_train_dict("data/fmda_nw_202401-05_f05.pkl", features_all=['Ed', 'Ew', 'solar', 'wind', 'elev', 'lon', 'lat', 'soilm', 'canopyw', 'groundflux', 'rain'])

In [ ]:
train = build_train_dict("data/test_CA_202401.pkl", features_all=['Ed', 'Ew', 'solar', 'wind', 'elev', 'lon', 'lat', 'soilm', 'canopyw', 'groundflux', 'rain'])

In [ ]:
train2 = build_train_dict("data/test_CA_202401.pkl", atm="RAWS", features_all=['Ed', 'Ew', 'solar', 'wind', 'elev', 'lon', 'lat', 'soilm', 'canopyw', 'groundflux', 'rain'])

In [ ]:
train2.keys()

In [ ]:
train2['CRVC1_202401']['features_list']

In [ ]:
train2['CRVC1_202401']['X'].shape

In [ ]:
import importlib
import data_funcs
importlib.reload(data_funcs)
from data_funcs import process_train_dict

In [ ]:
data_params.update({'hours': 168})

In [ ]:
train_h = process_train_dict("data/test_CA_202401.pkl", atm_dict="HRRR", data_params=data_params)

In [ ]:
train_r = process_train_dict("data/test_CA_202401.pkl", atm_dict="RAWS", data_params=data_params)

In [ ]:
train['CNFC1_202401'].keys()

In [ ]:
train_h['CNFC1_202401'].keys()

## Filter

In [ ]:
import importlib
import data_funcs
importlib.reload(data_funcs)
from data_funcs import split_timeseries, flag_lag_stretches, discard_keys_with_short_y, flag_dict_keys

In [ ]:
data_params.update({
    'max_intp_time': 12,
    'zero_lag_threshold': 12
})

In [ ]:
d = split_timeseries(train, hours=data_params['hours'], verbose=True)

In [ ]:
len(d.keys())

In [ ]:
d = discard_keys_with_short_y(d, hours=109, verbose=True)

In [ ]:
len(d.keys())

In [ ]:
flags = flag_dict_keys(d, data_params['zero_lag_threshold'], data_params['max_intp_time'], 
                       max_y = data_params['max_fm'], min_y = data_params['min_fm'])

In [ ]:
flags

In [ ]:
cases = list([*d.keys()])
flagged_cases = [element for element, flag in zip(cases, flags) if flag == 1]
print(len(flagged_cases))

In [ ]:
flagged_cases

In [ ]:
def remove_key_list(d, ls, verbose=False):
    for key in ls:
        if key in d:
            if verbose:
                print(f"Removing key {key} due to data flags")
            del d[key]

In [ ]:
remove_key_list(d, flagged_cases, verbose=True)